In [ ]:
import pandas as pd
import numpy as np
import os
import scipy
import itertools

import matplotlib.pyplot as plt    
import xgboost as xgb
iv=['np'    , 'os'     ,'pd'    , 'plt'     ,'tqdm'    , 'train'     ,'train_date'      , 'train_num' ,'data'    ]
def memchk(iv):
    import sys,operator
    ipython_vars=iv
    ,'warnings'    , 'xgboost']
    print sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)
    return sorted([{i:sys.getsizeof(globals().get(i))/1000000.0} for i in ipython_vars])
%matplotlib inline
from sklearn.metrics import confusion_matrix
import warnings
%load_ext autotime
warnings.filterwarnings('ignore')
from tqdm import tqdm
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))
plt.style.use('classic')

In [ ]:
def splitDatetime(df) :
    cols=df.select_dtypes(include = ['datetime64']).columns.values.tolist()
    for i in cols:
        df[str(i)+'year'] = df[i].apply(lambda l: l.year)
        df[str(i)+'month'] = df[i].apply(lambda l: l.month)
        df[str(i)+'day'] = df[i].apply(lambda l: l.day)
        df[str(i)+'hour'] = df[i].apply(lambda l: l.hour)
        df[str(i)+'date'] = df[i].apply(lambda l: l.date())
    return df

In [ ]:
def TTDistplot(train,test,Col,plt,loc,size=(2,3)):
    
    ax1 = plt.subplot2grid(size, loc)
    train[Col].value_counts().plot(kind='kde', color='#FA2379', label='train', alpha=alpha)
    test[Col].value_counts().plot(kind='kde', label='test', alpha=alpha)
    ax1.set_xlabel(Col)
    ax1.set_title("What's the distribution of " +Col)
    plt.legend(loc='best')
    #plt.tight_layout()

In [ ]:
def dplot():
    plt.rc('font', size=13)
    fig = plt.figure(figsize=(18, 8))
    alpha = 0.6
    return plt

In [ ]:
train = pd.read_csv(r'C:\Users\sushant\Documents\Datasets\NYC\train.csv')
train_len=len(train)
print train_len
train.info()

In [ ]:
print 'Categorical'
cat_var= train.select_dtypes(include = ['object']).columns.values.tolist()
print cat_var
print '\nNumerical Variables'
num_var = train.select_dtypes(include = ['int64','float64']).columns.values.tolist()
print num_var

In [ ]:
train[num_var].head()

In [ ]:
train[cat_var].head()

In [ ]:
test = pd.read_csv(r'C:\Users\sushant\Documents\Datasets\NYC\test.csv')

In [ ]:
data=pd.concat([train,test])


In [ ]:
del train,test


In [ ]:
data.info()

In [ ]:
data['pickup_datetime']=data['pickup_datetime'].astype('datetime64')


In [ ]:
data['dropoff_datetime']= pd.to_datetime(data['dropoff_datetime'], format = '%Y-%m-%d %H:%M:%S')

In [ ]:
data=splitDatetime(data)

In [ ]:
data.head()

In [ ]:
print 'Categorical'
cat_var= data.select_dtypes(include = ['object']).columns.values.tolist()
print cat_var
print '\nNumerical Variables'
num_var = data.select_dtypes(include = ['int64','float64']).columns.values.tolist()
print num_var

In [ ]:
data.info()

In [ ]:
data[['passenger_count', 'store_and_fwd_flag']].head()

# '''''''''''''''''''''''
# Exploration
# ....................

In [ ]:
printmd('### '+ str(len(data.columns)))
print data.columns.values

In [ ]:
data[cat_var].describe()

In [ ]:
feat=['dropoff_datetime',
  'dropoff_latitude',
  'dropoff_longitude',
  'id',
 'passenger_count',
 'pickup_datetime',
 'pickup_latitude',
 'pickup_longitude',
 'store_and_fwd_flag',
 'trip_duration',
 'vendor_id',
 'dropoff_datetimeyear',
 'dropoff_datetimemonth',
 'dropoff_datetimeday',
 'dropoff_datetimehour',
 'dropoff_datetimedate',
 'pickup_datetimeyear',
 'pickup_datetimemonth',
 'pickup_datetimeday',
 'pickup_datetimehour',
 'pickup_datetimedate']

In [ ]:
data['passenger_count'].iloc[:train_len].hist()
data['passenger_count'].iloc[train_len:].hist()

In [ ]:
.

In [ ]:
train=data.iloc[:train_len]
test=data.iloc[train_len:]

In [ ]:
dp=dplot()
loc=(0,0)
size=(2,3) #(size suitable for 3 columns 2 rows)
alpha=0.6
TTDistplot(train,test,'passenger_count',dp,loc,size)

In [ ]:
data['trip_duration'].value_counts().plot(kind='kde',title='Distribution of trip duration')

In [ ]:
data[['trip_duration','pickup_datetimehour']].groupby('pickup_datetimehour').sum().plot(y='trip_duration',kind='line')
data[['trip_duration','pickup_datetimeday']].groupby('pickup_datetimeday').sum().plot(y='trip_duration',kind='line')
data[['trip_duration','pickup_datetimemonth']].groupby('pickup_datetimemonth').sum().plot(y='trip_duration',kind='line')
#data[['trip_duration','pickup_datetimeyear']].groupby('pickup_datetimeyear').sum().plot(y='trip_duration',kind='line')

In [ ]:
data[['trip_duration','pickup_datetimemonth']].max()

In [ ]:
data['Train']=0
data['Train'].iloc[train.index]=1

In [ ]:
data.Train.value_counts()

In [ ]:
#data.to_csv(r'C:\Users\sushant\Documents\Datasets\NYC\data_clean.csv')